In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import os 
import sqlite3
from credentials import key
import re
import numpy as np

In [2]:
# # api call for first half of results

# # set up the request parameters
# params_1 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'page': '1',
#     'max_page': '5'
# }

# # make the http GET request to RedCircle API
# api_result_1 = requests.get('https://api.redcircleapi.com/request', params_1)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_1.json()))
# json_data_1 = api_result_1.json()

('{"request_info": {"success": true, "credits_used": 35, "credits_remaining": '
 '65, "credits_used_this_request": 5}, "request_parameters": {"search_term": '
 '"laundry detergent", "category_id": "uqe9q", "type": "search", "page": "1", '
 '"max_page": "5"}, "request_metadata": {"created_at": '
 '"2023-05-07T00:42:45.542Z", "processed_at": "2023-05-07T00:43:08.943Z", '
 '"total_time_taken": 23.4, "pages": [{"created_at": '
 '"2023-05-07T00:42:45.542Z", "processed_at": "2023-05-07T00:42:46.871Z", '
 '"total_time_taken": 1.33, "page": "1", "target_url": '
 '"https://www.target.com/s?searchTerm=laundry+detergent&Nao=0&category=uqe9q"}, '
 '{"created_at": "2023-05-07T00:42:46.872Z", "processed_at": '
 '"2023-05-07T00:42:52.315Z", "total_time_taken": 5.44, "page": 2, '
 '"target_url": '
 '"https://www.target.com/s?searchTerm=laundry+detergent&Nao=24&category=uqe9q"}, '
 '{"created_at": "2023-05-07T00:42:52.315Z", "processed_at": '
 '"2023-05-07T00:43:05.544Z", "total_time_taken": 13.23, "pa

In [3]:
# # api call for second half of results


# # set up the request parameters
# params_2 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'max_page': '8',
#     'page': '6'
# }


# # make the http GET request to RedCircle API
# api_result_2 = requests.get('https://api.redcircleapi.com/request', params_2)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_2.json()))
# json_data_2 = api_result_2.json()

('{"request_info": {"success": true, "credits_used": 38, "credits_remaining": '
 '62, "credits_used_this_request": 3}, "request_parameters": {"search_term": '
 '"laundry detergent", "category_id": "uqe9q", "type": "search", "max_page": '
 '"8", "page": "6"}, "request_metadata": {"created_at": '
 '"2023-05-07T00:53:29.447Z", "processed_at": "2023-05-07T00:53:38.588Z", '
 '"total_time_taken": 9.14, "pages": [{"created_at": '
 '"2023-05-07T00:53:29.447Z", "processed_at": "2023-05-07T00:53:31.001Z", '
 '"total_time_taken": 1.55, "page": "6", "target_url": '
 '"https://www.target.com/s?searchTerm=laundry+detergent&Nao=120&category=uqe9q"}, '
 '{"created_at": "2023-05-07T00:53:31.002Z", "processed_at": '
 '"2023-05-07T00:53:33.146Z", "total_time_taken": 2.14, "page": 7, '
 '"target_url": '
 '"https://www.target.com/s?searchTerm=laundry+detergent&Nao=144&category=uqe9q"}, '
 '{"created_at": "2023-05-07T00:53:33.146Z", "processed_at": '
 '"2023-05-07T00:53:38.588Z", "total_time_taken": 5.44, "

In [4]:
# # save the 1st search results json in a .json file
# out_file = open("resources/search-results-1.json", "w")
  
# json.dump(json_data_1, out_file, indent = 4)
  
# out_file.close()

# # save the 2nd search results json in a .json file
# out_file = open("resources/search-results-2.json", "w")
  
# json.dump(json_data_2, out_file, indent = 4)
  
# out_file.close()

In [5]:
# read the 2 json search results as dicts

# read the first json files
f = open('resources/search-results-1.json')
# returns JSON object as a dictionary
search_data_1 = json.load(f)
# Closing file
f.close()

# read the first json files
f = open('resources/search-results-2.json')
# returns JSON object as a dictionary
search_data_2 = json.load(f)
# Closing file
f.close()

In [6]:
# create dataframe from 2 search results dicts
search_results_df_1 = pd.DataFrame(search_data_1['search_results'])
search_results_df_2 = pd.DataFrame(search_data_2['search_results'])
search_results_df = search_results_df_1.append(search_results_df_2, ignore_index=True)

search_results_df.head()

,position,product,offers,fulfillment,page,position_overall,seller
0,1,{'title': 'Tide Plus Ultra Oxi Liquid Laundry ...,"{'primary': {'price': 19.99, 'currency': 'USD'...",{'type': '1p'},1,1,NaN
1,2,{'title': 'Tide High Efficiency Liquid Laundry...,"{'primary': {'price': 19.99, 'currency': 'USD'...",{'type': '1p'},1,2,NaN
2,3,{'title': 'All Free Clear Liquid Concentrated ...,"{'primary': {'price': 13.29, 'currency': 'USD'...",{'type': '1p'},1,3,NaN
3,4,{'title': 'Gain + Aroma Boost Original Scent H...,"{'primary': {'price': 12.99, 'currency': 'USD'...",{'type': '1p'},1,4,NaN
4,5,{'title': 'Free Clear HE Liquid Laundry Deterg...,"{'primary': {'price': 7.99, 'currency': 'USD',...",{'type': '1p'},1,5,NaN


In [7]:
# Each column is a dictionary. Get the dictionary of product data from the 'product' column.
product_json = search_results_df.loc[8]['product']

# Each column is a dictionary. Get the dictionary of price data from the 'offers'column.
offers_json = search_results_df.loc[8]['offers']
# The price information is in the 'primary' key of the offers_json
price_json = offers_json['primary']
pprint(product_json)

{'brand': 'Seventh Generation',
 'brand_link': 'https://www.target.com/b/seventh-generation/-/N-5vidi',
 'class_id': 2,
 'department_id': 3,
 'dpci': '003-02-1552',
 'feature_bullets': ['Seventh Generation Laundry Detergent lifts stubborn '
                     'stains like tomato sauce, grass and coffee with our '
                     'enzyme rich formula',
                     'THis liquid detergent is free of dyes, synthetic '
                     'fragrances & artificial brighteners',
                     'Don’t just take our word for it, our washing detergent '
                     'is a USDA Certified Biobased Product 97%',
                     'Our laundry detergent is designed to work in both HE and '
                     'standard machines, it’s powerful in all washing '
                     'temperatures, and its septic safe',
                     'The Fresh Citrus scent in our detergent liquid is '
                     'derived from 100% essential oils and botanical extracts

In [8]:
# clean up the product dictionary for only what is needed
product_json = {
    'title': product_json['title'],
    'link': product_json['link'],
    'department_id': product_json['department_id'],
    'brand': product_json['brand'],
    'rating':product_json['rating'],
    'tcin': product_json['tcin'],
    'position': search_results_df.loc[8]['position']
}
product_json

{'title': 'Seventh Generation Liquid Laundry Detergent Soap - Fresh Citrus Scent - 60 Loads - 90 fl oz',
 'link': 'https://www.target.com/p/seventh-generation-liquid-laundry-detergent-soap-fresh-citrus-scent-60-loads-90-fl-oz/-/A-84215097',
 'department_id': 3,
 'brand': 'Seventh Generation',
 'rating': 4.5,
 'tcin': '84215097',
 'position': 9}

In [9]:
# For each row of the search_results_df, make a dict for product_json and price_json and add it to the full_product_df.
price_x_product_df = pd.DataFrame(

                        {'title': pd.Series(dtype='str'),
                        'link': pd.Series(dtype='str'),
                        'department_id': pd.Series(dtype='int'),
                        'brand': pd.Series(dtype='str'),
                        'rating': pd.Series(dtype='float'),
                        'tcin': pd.Series(dtype='str'),
                        'price': pd.Series(dtype='float'),
                        'currency': pd.Series(dtype='str'),
                        'symbol': pd.Series(dtype='str'),
                        'sponsored': pd.Series(dtype='str')
                        }
                    )

for i in range(len(search_results_df)):

    product_json = search_results_df.loc[i]['product']
    price_json = search_results_df.loc[i]['offers']['primary']
    position = search_results_df.loc[i]['position']
    if 'rating' in product_json:
        r = product_json['rating']
    else:
        r = 0
    if 'sponsored' in product_json:
        s = product_json['sponsored']
    else:
        s = 'False'

    full_product_info = {
        'title': product_json['title'],
        'link': product_json['link'],
        'department_id': product_json['department_id'],
        'brand': product_json['brand'],
        'rating': r,
        'tcin': product_json['tcin'],
        'price': price_json['price'], 
        'currency': price_json['currency'], 
        'symbol': price_json['symbol'],
        'sponsored': s
    }

    price_x_product_df = price_x_product_df.append(full_product_info, ignore_index=True)

price_x_product_df


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored
0,Tide Plus Ultra Oxi Liquid Laundry Detergent -...,https://www.target.com/p/tide-plus-ultra-oxi-l...,3,Tide,0.0,87469579,19.99,USD,$,True
1,Tide High Efficiency Liquid Laundry Detergent ...,https://www.target.com/p/tide-high-efficiency-...,3,Tide,0.0,87469585,19.99,USD,$,True
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False
3,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,50237771,12.99,USD,$,False
4,Free Clear HE Liquid Laundry Detergent - 100 f...,https://www.target.com/p/free-clear-he-liquid-...,3,up & up,0.0,13186521,7.99,USD,$,False
...,...,...,...,...,...,...,...,...,...,...
197,Zum Laundry Soap - Lavender - 64 fl oz,https://www.target.com/p/zum-laundry-soap-lave...,3,Zum,0.0,87559686,14.49,USD,$,False
198,Zero Odor Laundry Odor Eliminator Additive -16oz,https://www.target.com/p/zero-odor-laundry-odo...,3,Zero Odor,0.0,89085844,14.99,USD,$,False
199,"Grab Green Baby Laundry Detergent Pods, 50 Pod...",https://www.target.com/p/grab-green-baby-laund...,3,Grab Green,0.0,88267112,18.25,USD,$,False
200,Noodle &#38; Boo Ultra Safe Laundry Detergent ...,https://www.target.com/p/noodle-38-boo-ultra-s...,7,Noodle & Boo,5.0,88361852,17.99,USD,$,False


In [10]:
# find duplicated titles
s = pd.Series(price_x_product_df.duplicated(subset=['title'], keep=False))
price_x_product_df2 = price_x_product_df.copy()
price_x_product_df2['duplicate_title'] = s
# print the duplicate title rows
price_x_product_df2 = price_x_product_df2.loc[price_x_product_df2['duplicate_title'] == True]
price_x_product_df2

,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,duplicate_title
8,Seventh Generation Liquid Laundry Detergent So...,https://www.target.com/p/seventh-generation-li...,3,Seventh Generation,4.5,84215097,12.99,USD,$,True,True
18,Persil Original Scent Liquid Laundry Detergent...,https://www.target.com/p/persil-original-scent...,3,Persil,4.4,50302226,13.99,USD,$,False,True
19,Tide Plus Febreze High Efficiency Liquid Laund...,https://www.target.com/p/tide-plus-febreze-hig...,3,Tide,0.0,15043040,12.99,USD,$,False,True
20,Purex with Crystals Fragrance Lavender Blossom...,https://www.target.com/p/purex-with-crystals-f...,3,Purex,4.7,17206874,9.19,USD,$,False,True
21,Free &#38; Clear Liquid Laundry Detergent - 10...,https://www.target.com/p/free-38-clear-liquid-...,3,Everspring,4.6,75663163,11.99,USD,$,False,True
22,Tide with Downy Powder Laundry Detergent - 148oz,https://www.target.com/p/tide-with-downy-powde...,3,Tide,4.7,52928047,22.99,USD,$,False,True
23,Laundry Detergent - Lemon &#38; Mint - 100 fl ...,https://www.target.com/p/laundry-detergent-lem...,3,Everspring,4.6,77678339,11.99,USD,$,False,True
27,Tide Pods Laundry Detergent Pacs - Spring Mead...,https://www.target.com/p/tide-pods-laundry-det...,3,Tide,0.0,51086522,7.99,USD,$,False,True
29,Seventh Generation EasyDose Ultra Concentrated...,https://www.target.com/p/seventh-generation-ea...,3,Seventh Generation,4.6,76158734,13.99,USD,$,True,True
30,Seventh Generation Liquid Laundry Detergent So...,https://www.target.com/p/seventh-generation-li...,3,Seventh Generation,0.0,84215100,13.69,USD,$,True,True


In [11]:
price_x_product_df2.groupby(by=['title', 'sponsored']).count()

,,link,department_id,brand,rating,tcin,price,currency,symbol,duplicate_title
title,sponsored,,,,,,,,,
9 Elements Liquid Laundry Detergent - Eucalyptus - 92 fl oz,False,2,2,2,2,2,2,2,2,2
Ariel Powder Laundry Detergent - 211oz,False,2,2,2,2,2,2,2,2,2
Arm &#38; Hammer Plus OxiClean with Odor Blasters - 42ct/29.6oz,False,2,2,2,2,2,2,2,2,2
ECOS Plastic-Free Laundry Detergent Sheets - Free &#38; Clear - 7.9oz/64 Loads,False,2,2,2,2,2,2,2,2,2
ECOS Plastic-Free Laundry Detergent Sheets - Magnolia &#38; Lily - 7.9oz/64 Loads,True,5,5,5,5,5,5,5,5,5
Free &#38; Clear Liquid Laundry Detergent - 100 fl oz - Everspring&#8482;,False,2,2,2,2,2,2,2,2,2
Fresh Linen HE Liquid Laundry Detergent 150 fl oz - up &#38; up&#8482;,False,2,2,2,2,2,2,2,2,2
Laundry Detergent - Lemon &#38; Mint - 100 fl oz - Everspring&#8482;,False,2,2,2,2,2,2,2,2,2
Odor Eliminator Liquid Laundry Detergent - Restoring Fresh - 92oz - up &#38; up&#8482;,False,2,2,2,2,2,2,2,2,2


In [12]:
# all the sponsored rows are duplicates, so drop them
price_x_product_df = price_x_product_df.loc[price_x_product_df['sponsored'] != True]
price_x_product_df = price_x_product_df.sort_values('title')

ounces

In [13]:
temp =[]
primary_title = []
ounces = []

# separate the title from the other details
for x in price_x_product_df['title']:
    i = x.split('-')
    temp.append(i)

#kep track of number of rows with no size.
no_size = 0

# put the title and the oz/fl oz in separate lists
for t in temp:

    primary_title.append(t[0])

    size = ''

    for x in t:

        if re.search('oz', x):
            size = x
    
    if size == '':
        no_size += 1

    ounces.append(size)
print('Rows with no ounces:')
print(no_size)
    
# add these lists to the search results df
price_x_product_df['primary_title'] = primary_title
price_x_product_df['ounces'] = ounces
price_x_product_df.head(15)


Rows with no ounces:
36


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
194,9 Elements LQ Laundry Detergent - Lavender - 6...,https://www.target.com/p/9-elements-lq-laundry...,3,9 Elements,0.0,87559729,13.39,USD,$,False,9 Elements LQ Laundry Detergent,65 fl oz
130,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92 fl oz
131,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92 fl oz
133,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92 fl oz
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141 fl oz
25,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141 fl oz
163,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,https://www.target.com/p/all-fresh-clean-oxi-o...,3,All,5.0,87559719,12.99,USD,$,False,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,
32,All Liquid Laundry Detergent - Free Clear for ...,https://www.target.com/p/all-liquid-laundry-de...,3,All,0.0,13219348,8.89,USD,$,False,All Liquid Laundry Detergent,88 fl oz
88,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.6,14044973,9.79,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,39ct/25.8oz
17,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.8,47022733,13.69,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,60ct/24.7oz


In [14]:
# x = price_x_product_df.iloc[7:8]
# xounces = x['ounces']

# for p in xounces:

#     temp_size = p.split('/')
#     print('temp_size:', temp_size)

#     for t in temp_size:
#         print(t)
#         if t.find('oz') != -1:
#             print('oz yes')
#             list = t.split("oz")
#             print("list: ", list)
#             ounces = list[0]
#             print(ounces)

In [15]:
# # make size col have just the number of ounces
# temp_ounces = []

# for x in price_x_product_df['ounces']:
    
#     temp_size = x.split('/') # [65oz, 23ct]

#     for t in temp_size:
#         if t.find('oz') != -1:
#             list = t.split("oz")
#             ounces = list[0].split("fl")[0]


#     temp_ounces.append(ounces)

# for i in temp_ounces:
#     print(i)
# price_x_product_df['ounces'] = temp_ounces
# print(len(price_x_product_df))
# price_x_product_df.head(15)

In [16]:
# make size col have just the number of ounces
temp_ounces = []

for x in price_x_product_df['ounces']:

    size = ''
    
    temp_size = x.split('/') # [65oz, 23ct]

    for t in temp_size:
        if re.search('oz', t):
            size_list = re.findall("\d+", t)
            if len(size_list) <= 1:
                size = size_list[0]
            else:
                size = size_list[0] + '.' + size_list[1]


    temp_ounces.append(size)


price_x_product_df['ounces'] = temp_ounces
print(len(price_x_product_df))
price_x_product_df.head(15)

187


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
194,9 Elements LQ Laundry Detergent - Lavender - 6...,https://www.target.com/p/9-elements-lq-laundry...,3,9 Elements,0.0,87559729,13.39,USD,$,False,9 Elements LQ Laundry Detergent,65
130,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92
131,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92
133,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141
25,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141
163,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,https://www.target.com/p/all-fresh-clean-oxi-o...,3,All,5.0,87559719,12.99,USD,$,False,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,
32,All Liquid Laundry Detergent - Free Clear for ...,https://www.target.com/p/all-liquid-laundry-de...,3,All,0.0,13219348,8.89,USD,$,False,All Liquid Laundry Detergent,88
88,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.6,14044973,9.79,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,25.8
17,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.8,47022733,13.69,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,24.7


In [17]:
# convert the blank ounces to nan and drop those rows
price_x_product_df['ounces'] = price_x_product_df['ounces'].replace('',np.NaN)
price_x_product_df = price_x_product_df.dropna()
print(len(price_x_product_df))
price_x_product_df.head()

151


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
194,9 Elements LQ Laundry Detergent - Lavender - 6...,https://www.target.com/p/9-elements-lq-laundry...,3,9 Elements,0.0,87559729,13.39,USD,$,False,9 Elements LQ Laundry Detergent,65
130,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92
131,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92
133,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141


In [18]:
price_x_product_df.dtypes

title             object
link              object
department_id      int64
brand             object
rating           float64
tcin              object
price            float64
currency          object
symbol            object
sponsored         object
primary_title     object
ounces            object
dtype: object

In [19]:
# price_x_product_df['ounces'] = price_x_product_df['ounces'].astype(float)
# price_x_product_df.dtypes


In [20]:
# save the search results in csv
os.makedirs('resources', exist_ok=True)  
price_x_product_df.to_csv('resources/price-x-product-data.csv')  

In [21]:
# create sqlite db
# conn = sqlite3.connect(':memory:')

conn = sqlite3.connect('laundry_detergent.db')

c = conn.cursor()

conn.execute("DROP TABLE laundry_detergents")

# create able
c.execute("""CREATE TABLE laundry_detergents(
        title TEXT,
        link TEXT,
        department_id INTEGER,
        brand TEXT,
        rating FLOAT,
        tcin TEXT,
        price REAL,
        currency TEXT,
        symbol TEXT,
        sponsored INTEGER,
        primary_title TEXT,
        ounces INTEGER
    )""")

# add the price-x-product-data.csv data to table
data = pd.read_csv('resources/price-x-product-data.csv')
data.to_sql('laundry_detergents', conn, if_exists='replace', index=False)
# _________________________
# queries
# _________________________

c.execute("""
    SELECT t.c, t.primary_title FROM
        (SELECT count(primary_title) as c, primary_title
        FROM laundry_detergents
        GROUP BY primary_title) as t
    WHERE t.c > 1
    """)

# c.execute("""
#     SELECT * FROM laundry_detergents
# """)

# _________________________
# end queries
# _________________________

c.execute
rows = c.fetchall()

conn.commit()

conn.close()

# check if any rows are not laundry detergent by seeing if the word in product title.
# for row in rows:
#     if 'Detergent' not in row[0]:
#         print(row[0])

for row in rows:
    print(row)

(2, '9 Elements Liquid Laundry Detergent ')
(2, 'All Mighty Pacs Free Clear Laundry Detergent Pacs ')
(2, 'Ariel Powder Laundry Detergent ')
(2, 'Arm &#38; Hammer Plus OxiClean with Odor Blasters ')
(3, 'ECOS Plastic')
(2, 'Free &#38; Clear Liquid Laundry Detergent ')
(2, 'Fresh Linen HE Liquid Laundry Detergent 150 fl oz ')
(2, 'Laundry Detergent ')
(2, 'Odor Eliminator Liquid Laundry Detergent ')
(2, 'Persil Discs Laundry Detergent Pacs Original ')
(2, 'Persil Original Liquid Concentrated Laundry Detergent ')
(2, 'Persil Original Scent Liquid Laundry Detergent ')
(2, 'Purex with Crystals Fragrance Lavender Blossom Liquid Laundry Detergent ')
(2, 'Purex with Oxi Liquid Laundry Detergent ')
(2, 'Seventh Generation Liquid Laundry Detergent Soap ')
(2, 'Seventh Generation Power + Liquid Laundry Detergent Soap Clean Scent ')
(2, 'Tide Hygienic Clean Heavy Duty Power Pods Laundry Detergent Pacs ')
(3, 'Tide Liquid Clean Laundry Detergent ')
(3, 'Tide Plus Febreze High Efficiency Liquid Lau

/Users/annaweeks/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
